# Деплой Django додатку (Production)

## Мета лекції

Після цієї лекції ви:

* зрозумієте різницю між development та production;
* знатимете основні етапи деплою Django-додатку;
* навчитеся готувати Django до production;
* отримаєте уявлення про Gunicorn, Nginx, Docker;
* зрозумієте типові помилки при деплої.

---

## 1. Що таке деплой і навіщо він потрібен

**Деплой** — це процес розгортання додатку на сервері, щоб ним могли користуватися реальні користувачі.

### Development vs Production

| Development         | Production         |
| ------------------- | ------------------ |
| DEBUG = True        | DEBUG = False      |
| Локальний сервер    | Реальний сервер    |
| SQLite              | PostgreSQL / MySQL |
| manage.py runserver | Gunicorn / uWSGI   |

📌 **Django не призначений для роботи в production без додаткових компонентів**

---

## 2. Типова архітектура Django в production

```
Client (Browser)
        ↓
     Nginx
        ↓
   Gunicorn
        ↓
     Django
        ↓
    Database
```

* **Nginx** — вебсервер (статика, проксі)
* **Gunicorn** — WSGI сервер
* **Django** — backend логіка

---

## 3. Підготовка Django до production

### settings.py — ключові налаштування

#### DEBUG

```python
DEBUG = False
```

❗ Ніколи не деплойте з `DEBUG = True`

---

#### ALLOWED_HOSTS

```python
ALLOWED_HOSTS = ['example.com', 'www.example.com']
```

---

#### SECRET_KEY

❌ Погано:

```python
SECRET_KEY = 'hardcoded'
```

✅ Добре:

```python
import os
SECRET_KEY = os.environ.get('SECRET_KEY')
```

---

## 4. Статичні файли

### settings.py

```python
STATIC_URL = '/static/'
STATIC_ROOT = BASE_DIR / 'staticfiles'
```

### Збірка статики

```bash
python manage.py collectstatic
```

📌 Nginx обслуговує static, не Django

---

## 5. Віртуальне середовище та залежності

```bash
python -m venv venv
source venv/bin/activate
pip install -r requirements.txt
```

### requirements.txt

```text
django
gunicorn
djangorestframework
```

---

## 6. Gunicorn — WSGI сервер

### Запуск Gunicorn

```bash
gunicorn project_name.wsgi:application
```

📌 Gunicorn замінює `runserver`

---

## 7. Nginx — вебсервер

### Приклад конфігурації

```nginx
server {
    server_name example.com;

    location /static/ {
        alias /var/www/project/staticfiles/;
    }

    location / {
        proxy_pass http://127.0.0.1:8000;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

---

## 8. База даних у production

❌ Не рекомендовано:

* SQLite

✅ Рекомендовано:

* PostgreSQL
* MySQL

### Приклад (PostgreSQL)

```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'dbname',
        'USER': 'dbuser',
        'PASSWORD': os.environ.get('DB_PASSWORD'),
        'HOST': 'localhost',
        'PORT': '5432',
    }
}
```

---

## 9. Міграції

```bash
python manage.py migrate
python manage.py createsuperuser
```

---

## 10. Безпека (коротко)

```python
SECURE_SSL_REDIRECT = True
CSRF_COOKIE_SECURE = True
SESSION_COOKIE_SECURE = True
```

📌 HTTPS — must-have

---

## 11. Docker (огляд)

### Dockerfile (спрощено)

```dockerfile
FROM python:3.11
WORKDIR /app
COPY . .
RUN pip install -r requirements.txt
CMD gunicorn project.wsgi:application --bind 0.0.0.0:8000
```

📌 Docker спрощує деплой та масштабування

---

## 12. Хостинги для Django

* VPS (DigitalOcean, Hetzner, AWS)
* PaaS (Render, Railway, Fly.io)

📌 Для навчання — PaaS простіше

---

## 13. Типові помилки

❌ DEBUG = True

❌ Не зібрали static

❌ SECRET_KEY у репозиторії

❌ Використання runserver

---

## 14. Коротке резюме

* Деплой = production
* Gunicorn + Nginx — класика
* DEBUG = False
* collectstatic — обовʼязково
* Docker — сучасний підхід

---

## Практичне завдання

1. Підготувати settings.py для production
2. Зібрати static
3. Запустити проєкт через Gunicorn
4. (Опціонально) зібрати Docker image